<a href="https://colab.research.google.com/github/HWAN-OH/H2-Ammonia-Simulator/blob/main/main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# main.py
import config
import calculator
import reporter

def run_simulation():
    """LCOA 시뮬레이션을 실행하고 결과를 저장합니다."""
    print("--- Starting LCOA Simulation for Green Ammonia ---")

    # 1. 연간 생산량 계산
    annual_h2_kg, annual_nh3_tonne = calculator.calculate_annual_production()
    print(f"Annual Ammonia Production: {annual_nh3_tonne:,.2f} tonnes")

    # 2. CAPEX 계산
    capex_costs = calculator.calculate_capital_costs()
    print(f"Total CAPEX (incl. replacement): ${capex_costs['total_capex_with_replacement']:,.2f}")

    # 3. OPEX 계산
    opex_costs = calculator.calculate_annual_operating_costs(capex_costs)
    print(f"Total Annual OPEX: ${opex_costs['total_annual_opex']:,.2f}")

    # 4. LCOA 계산
    lcoa_results = calculator.calculate_lcoa(
        capex_costs['total_capex_with_replacement'],
        opex_costs['total_annual_opex'],
        annual_nh3_tonne
    )
    print(f"Final LCOA at demand center: ${lcoa_results['lcoa_final']:,.2f} per tonne")

    # 5. 모든 결과 취합
    final_results = {
        "Plant Capacity (MW)": config.PLANT_CAPACITY_MW,
        "Capacity Factor (%)": config.CAPACITY_FACTOR * 100,
        "Electricity Cost ($/kWh)": config.ELECTRICITY_COST,
        "Discount Rate (%)": config.DISCOUNT_RATE * 100,
        "Annual NH3 Production (tonne)": annual_nh3_tonne,
        **{f"CAPEX - {k.replace('_', ' ').title()} ($)": v for k, v in capex_costs.items()},
        **{f"OPEX - {k.replace('_', ' ').title()} ($)": v for k, v in opex_costs.items()},
        **{f"LCOA - {k.replace('_', ' ').title()} ($/tonne)": v for k, v in lcoa_results.items()},
    }

    # 6. 결과 저장
    reporter.save_results_to_csv(final_results)

    print("--- Simulation Finished ---")


if __name__ == "__main__":
    run_simulation()